# Real Dataset Experiment (German Credit Dataset-- Group Fairness)
Generate the dataset for LTR setting


In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib import colors, ticker, cm

from pprint import pprint
from progressbar import progressbar

%matplotlib inline

In [2]:
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'png')
plt.rcParams['savefig.dpi'] = 75

plt.rcParams['figure.autolayout'] = False
plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 14

plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = "serif"
plt.rcParams['font.serif'] = "cm"
plt.rcParams['text.latex.preamble'] = r"\usepackage{subdepth}, \usepackage{type1cm}"

/tmp/ipykernel_29551/1285856066.py:2: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('pdf', 'png')


In [3]:
cd ..

/home/ramon


In [4]:
from train_yahoo_dataset import on_policy_training
from YahooDataReader import YahooDataReader
import torch
from models import NNModel, LinearModel
from evaluation import evaluate_model

In [5]:
# read data 
import pickle as pkl
dr = YahooDataReader(None)
dr.data = pkl.load(open("GermanCredit/german_train_rank.pkl", "rb"))
vdr = YahooDataReader(None)
vdr.data = pkl.load(open("GermanCredit/german_test_rank.pkl","rb"))

FileNotFoundError: [Errno 2] No such file or directory: 'GermanCredit/german_train_rank.pkl'

In [ ]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
args = Namespace(conditional_model=True, gpu_id=None, progressbar=True, evaluate_interval=250, input_dim=29, 
                 eval_rank_limit=1000,
                fairness_version="asym_disparity", entropy_regularizer=0.0, save_checkpoints=False, num_cores=1,
                pooling='concat_avg', dropout=0.0, hidden_layer=8, summary_writing=False, 
                 group_fairness_version="asym_disparity",early_stopping=False, lr_scheduler=False, 
                 validation_deterministic=False, evalk=1000, reward_type="ndcg", baseline_type="value", 
                 use_baseline=True, entreg_decay=0.0, skip_zero_relevance=True, eval_temperature=1.0, optimizer="Adam",
                clamp=False)
torch.set_num_threads(args.num_cores)
args.progressbar = False 

args.group_feat_id = 3

In [ ]:
model_params_list = []
lambdas_list = [0.0, 0.1, 1.0, 10.0, 12.0, 15.0, 20.0, 25.0, 50.0, 100.0]
plt_data_pl = np.zeros((len(lambdas_list)+1, 2))
for i, lgroup in enumerate(lambdas_list):
        torch.set_num_threads(args.num_cores)
        args.lambda_reward = 1.0
        args.lambda_ind_fairness = 0.0
        args.lambda_group_fairness = lgroup
        
        args.lr = 0.001
        args.epochs = 10
        args.progressbar = False
        args.weight_decay = 0.0
        args.sample_size = 25
        args.optimizer = "Adam"

        model = LinearModel(D=args.input_dim)

        model = on_policy_training(dr, vdr, model, args=args)
        if i == 0:
            results = evaluate_model(model, vdr, fairness_evaluation=False, group_fairness_evaluation=True, 
                                 deterministic=True, args=args, num_sample_per_query=20)
            print(results)
            plt_data_pl[0] = [results["ndcg"], results["avg_group_asym_disparity"]]
        results = evaluate_model(model, vdr, fairness_evaluation=False, group_fairness_evaluation=True, 
                                 deterministic=False, args=args, num_sample_per_query=20)
        print(results)
        model_params_list.append(model.w.weight.data.tolist()[0])
        print("Learnt model for lambda={} has model weights as {}".format(lgroup, model_params_list[-1]))
        plt_data_pl[i+1] = [results["ndcg"], results["avg_group_asym_disparity"]]

In [ ]:
plt_data_pl

In [ ]:
from itertools import permutations 
from scipy.optimize import linprog

In [ ]:


def assign_groups(groups):
    G = [[], []]
    for i in range(len(groups)):
        G[groups[i]].append(i)
    return G


def fair_rank(relevances, groups ,lmda = 1):
    n = len(relevances)
    pos_bias = vvector(n)
    G = assign_groups(groups)
    n_g, n_i = 0, 0
    n_g += (len(G)-1)*len(G)
    n_c = n**2 + n_g 
    
    
    c = np.ones(n_c)
    c[:n**2] *= -1
    c[n**2:] *= lmda
    A_eq = []
    #For each Row
    for i in range(n):
        A_temp = np.zeros(n_c)
        A_temp[i*n:(i+1)*n] = 1
        assert(sum(A_temp)==n)
        A_eq.append(A_temp)
        c[i*n:(i+1)*n] *= relevances[i]

    #For each coloumn
    for i in range(n):
        A_temp = np.zeros(n_c)
        A_temp[i:n**2:n] = 1
        assert(sum(A_temp)==n)
        A_eq.append(A_temp)
        #Optimization 
        c[i:n**2:n] *= pos_bias[i] 
    b_eq = np.ones(n*2) 
    A_eq = np.asarray(A_eq)
    bounds = [(0,1) for _ in range(n**2)] + [(0,None) for _ in range(n_g+n_i)]
    
    
    A_ub = []
    b_ub = np.zeros(n_g)
    sum_rels = []
    for group in G:
        #Avoid devision by zero
        sum_rel = np.max([np.sum(np.asarray(relevances)[group]), 0.01])
        sum_rels.append(sum_rel)
    comparisons = list(permutations(np.arange(len(G)),2))
    j = 0
    for a,b in comparisons:
        f = np.zeros(n_c)
        if len(G[a]) > 0 and len(G[b])>0 and sum_rels[a]/len(G[a]) >= sum_rels[b]/len(G[b]):
            for i in range(n):
                tmp1 = len(G[a]) / sum_rels[a] if i in G[a] else 0 
                tmp2 = len(G[b]) / sum_rels[b] if i in G[b] else 0 
                #f[i*n:(i+1)*n] *= max(0, sign*(tmp1 - tmp2))
                f[i*n:(i+1)*n] =  (tmp1 - tmp2)
            for i in range(n):
                f[i:n**2:n] *= pos_bias[i]
            f[n**2+j] = -1
        j += 1
        A_ub.append(f)

    res = linprog(c, A_eq=A_eq, b_eq=b_eq, A_ub=A_ub, b_ub=b_ub, bounds=bounds, 
                                  method = "interior-point")#, options=dict(tol=1e-12),)
    if res.success is False:
        print("Constraint not satisfied!!")
    probabilistic_ranking = np.reshape(res.x[:n**2],(n,n))
    return probabilistic_ranking, res, res.fun

In [ ]:
from baselines import *

In [ ]:
# Linear regression
from sklearn import linear_model
model = linear_model.LinearRegression(fit_intercept=False, normalize=False)
feats, rel = dr.data
feats = np.array([item for sublist in feats for item in sublist])
rel = np.array([item for sublist in rel for item in sublist])
model.fit(feats, rel)
# predictions on validation
feats, rel = vdr.data
se_sum = 0
length = 0
predicted_rels = []
for i, query in enumerate(feats):
    rel_pred = model.predict(query)
    predicted_rels.append(rel_pred)
    se_sum += np.sum((rel_pred - rel[i])**2)
    length += len(rel[i])
print(se_sum/ length)

In [ ]:
# # for each lambda
# now for each query, estimate all relevances
# we find a fair ranking, find it's loss w.r.t true relevances
lmbdas = np.linspace(0, 0.2, 20)

plt_data = np.zeros((len(lmbdas), 2))

for j, lmbda in enumerate(lmbdas):
    test_losses = []
    test_ndcgs = []
    
    for i in range(len(predicted_rels)):
        true_rels = rel[i]
        pred_rels = predicted_rels[i]
        groups = np.array(feats[i][:, args.group_feat_id], dtype=np.int)
        n = len(groups)
        
        P, _, _ = fair_rank(pred_rels, groups, lmbda)
        
        test_loss = get_fairness_loss(P, true_rels, vvector(n), groups)
        test_losses.append(test_loss)
        
        test_ndcg = get_ndcg(P, true_rels, vvector(n))
        test_ndcgs.append(test_ndcg)
        
    plt_data[j] = [np.mean(test_ndcgs), np.mean(test_losses)]
        
        
    print("Lambda: {}, Average Test Fairness Loss: {}, Average Test NDCG: {}".format(lmbda, 
                                                     np.mean(test_losses), np.mean(test_ndcgs)))

plt.scatter(plt_data[:, 0], plt_data[:, 1])

In [ ]:
plt.scatter(plt_data[:, 0], plt_data[:, 1])
plt.scatter(plt_data_pl[:, 0], plt_data_pl[:, 1])


In [ ]:
from zehlike import *

args.lr = [0.001]
args.lambda_reward = 1.0
plt_data_4 = []
lambdas = [0.0, 1.0, 10, 100, 1000, 10000, 100000, 1000000]
args.weight_decay = [0.0]
args.epochs = [10]
disparities_mat = np.zeros((len(lambdas), 1))
ndcg_mat = np.zeros((len(lambdas), 1))
for i, lg in enumerate(lambdas):
    args.lambda_group_fairness = lg
    model = LinearModel(D=args.input_dim)
    model = demographic_parity_train(model, dr, vdr, vvector(200), args)
    results = evaluate_model(
            model,
            vdr,
            fairness_evaluation=False,
            group_fairness_evaluation=True,
            deterministic=True,
            args=args,
            num_sample_per_query=100)
    plt_data_4.append([results["ndcg"], results["avg_group_asym_disparity"]])
    ndcg_mat[i, 0], disparities_mat[i,0] = results["ndcg"], results["avg_group_asym_disparity"]

In [ ]:
plt_data_4 = np.array(plt_data_4)

In [ ]:
plt_data_4

In [ ]:
plt.scatter(plt_data_4[:, 0], plt_data_4[:, 1])


In [ ]:
def ndcg_vs_disparity_plot(plt_data_mats, names, join=False, ranges=None):
    plt.figure(figsize=(6.5, 4))
    if ranges:
        plt.xlim(ranges[0])
        plt.ylim(ranges[1])
    initialize_params()
    for i, plt_data_mat in enumerate(plt_data_mats):
        if not join:
            plt.scatter(
                plt_data_mats[i][:, 0],
                plt_data_mats[i][:, 1],
                marker="*",
                label=names[i])
        else:
            plt.plot(
                plt_data_mats[i][:, 0],
                plt_data_mats[i][:, 1],
                marker="*",
                linestyle='--',
                label=names[i])
    plt.legend(fontsize=14)
#     plt.title("Utility-Fairness trade-off",y=-0.30)
    plt.xlabel("NDCG")
    plt.ylabel(r'$\hat{\mathcal{D}}_{\rm group}$', fontsize=16)
    plt.grid()
    plt.savefig('./plots/german_tradeoff.pdf', bbox_inches='tight')
    plt.show()

In [ ]:
ndcg_vs_disparity_plot([plt_data, plt_data_pl,  plt_data_4], ["Post-Processing ($\lambda \in [0, 0.2]$)",
                      "Our Method ($\lambda \in [0,100]$ )", 
                      "Zehlike \& Castillo 2018 ($\lambda \in [0, 10^6]$)"], join=True, ranges=[[0.60, 0.85], [0.00, 0.054]])

In [ ]:
ndcg_vs_disparity_plot([plt_data, plt_data_pl], ["Post-Processing ($\lambda \in [0, 0.2]$)",
                      "Our Method ($\lambda \in [0,100]$ )"], join=True, ranges=[[0.60, 0.85], [0.00, 0.054]])